In [1]:
import os
import torch
import cv2
import shutil
import json

import numpy as np
import pandas as pd

from glob import glob
from tqdm import tqdm

from torchvision.io import read_image
from torchvision.ops import masks_to_boxes


Data from the Public Dataset BUSI contain images and semantic segmentation masks.

This script is to convert all the semantic segmentation masks to .json files in Labelme format (same as data provided by Sponsor)

# Seperate Images and Labels

In [2]:
data_path = "C:/Users/frank/OneDrive/Desktop/GT_Practicum/data/Dataset_BUSI_with_GT/malignant/"
data_list = glob(os.path.join(data_path, "*.png"))
len(data_list)

In [5]:
data_list[0].split('_')

['C:/Users/frank/OneDrive/Desktop/GT',
 'Practicum/data/Dataset',
 'BUSI',
 'with',
 'GT/malignant\\malignant (1).png']

In [13]:
for data_path in tqdm(data_list):
    if data_path.split('_')[-1] == 'mask.png':
        new_data_path = os.path.join('C:/Users/frank/OneDrive/Desktop/GT_Practicum/data/Dataset_BUSI_with_GT/malignant/masks/', os.path.basename(data_path))
        shutil.copy(data_path, new_data_path)
    else:
        new_data_path = os.path.join('C:/Users/frank/OneDrive/Desktop/GT_Practicum/data/Dataset_BUSI_with_GT/malignant/imgs/', os.path.basename(data_path))
        shutil.copy(data_path, new_data_path)

100%|██████████| 421/421 [00:00<00:00, 538.63it/s]


# Generate jsons using mask

In [2]:
image_list = glob(os.path.join('C:/Users/frank/OneDrive/Desktop/GT_Practicum/data/Dataset_BUSI_with_GT/benign/imgs/', "*.png"))
mask_list = glob(os.path.join('C:/Users/frank/OneDrive/Desktop/GT_Practicum/data/Dataset_BUSI_with_GT/benign/masks/', "*.png"))
image_list.sort()
mask_list.sort()

In [3]:
for mask_path in tqdm(mask_list):
    mask = read_image(mask_path)
    width, height = mask.shape[2], mask.shape[1]
    
    bool_mask = mask > 0
    box = masks_to_boxes(bool_mask)

    img_path = os.path.basename(mask_path).split('_')[0] + '.png'

    json_path = os.path.join('C:/Users/frank/OneDrive/Desktop/GT_Practicum/data/Dataset_BUSI_with_GT/benign/jsons/', img_path.split('.')[0] + '.json')

    if os.path.exists(json_path):
        with open(json_path, 'r') as f:
            label_dic = json.load(f)
            
        shapes = {"label" : "benign",
                    "points" : [
                            [float(box[0][1]), float(box[0][0])],
                            [float(box[0][3]), float(box[0][2])]
                    ],
                    "group_id": None,
                    "shape_type": "rectangle",
                    "flags": {}
                    }
        label_dic['shapes'].append(shapes)
        
    else:
        label_dic = {"version" : "5.0.1",
             "flags" : {},
             "shapes" : [],
             "imagePath" : img_path,
             "imageData": None,
             "imageHeight" : height,
             "imageWidth" : width
             }
        
        shapes = {"label" : "benign",
            "points" : [
                    [float(box[0][1]), float(box[0][0])],
                    [float(box[0][3]), float(box[0][2])]
            ],
            "group_id": None,
            "shape_type": "rectangle",
            "flags": {}
            }
        label_dic['shapes'].append(shapes)

    with open(json_path, 'w') as f:
        json.dump(label_dic, f)
    

100%|██████████| 454/454 [00:01<00:00, 260.62it/s]
